# Pregunta 1

In [12]:
using JuMP, CSV, DataFrames, XLSX
using Gurobi  # Cambié GLPK por Gurobi, ya que lo mencionas en tu enunciado inicial

# Carga los datos desde los archivos CSV
demand_df = CSV.read("Demand.csv", DataFrame)
generators_df = CSV.read("Generators.csv", DataFrame)
lines_df = CSV.read("Lines.csv", DataFrame)

# Potencia base de 100MVA
pot_base = 100

# Conjunto de nodos, tiempos, generadores y líneas
N = 1:9 # Nodos de 1 a 9
T = 1:6 # Periodos de tiempo de t = 1 a t = 6
G = 1:size(generators_df, 1) # Generadores
L = 1:size(lines_df, 1) # Líneas de transmisión

# Inicializa el modelo de optimización
model = Model(Gurobi.Optimizer)

# Variables de decisión
@variable(model, p[g in G, t in T] >= 0) # Potencia generada por generador g en tiempo t
@variable(model, theta[n in N, t in T]) # Ángulo del voltaje en nodo n en tiempo t

# Parámetros del modelo
demand = Dict((n, t) => demand_df[n, t+1]/pot_base for n in N for t in T)
pot_min = Dict(g => generators_df.PotMin[g]/pot_base for g in G)
pot_max = Dict(g => generators_df.PotMax[g]/pot_base for g in G)
gen_cost = Dict(g => generators_df.GenCost[g] for g in G)
ramp = Dict(g => generators_df.Ramp[g]/pot_base for g in G)
line_max = Dict(l => lines_df.PotMax[l]/pot_base for l in L)
b_susceptance = Dict(l => 1/lines_df.Imp[l] for l in L)

# Restricciones del modelo
# Satisfacción de la demanda en cada nodo y tiempo con nombre para restricciones
@constraint(model, demanda[n in N, t in T],
    sum(p[g, t] for g in G if generators_df.BarConexion[g] == n) - 
    sum(b_susceptance[l]*(theta[lines_df.BarIni[l], t] - theta[lines_df.BarFin[l], t]) for l in L if lines_df.BarIni[l] == n) -
    sum(b_susceptance[l]*(theta[lines_df.BarFin[l], t] - theta[lines_df.BarIni[l], t]) for l in L if lines_df.BarFin[l] == n) == demand[n, t])

# Límites de generación de cada generador
for g in G, t in T
    @constraint(model, pot_min[g] <= p[g, t] <= pot_max[g])
end

# Capacidad de las líneas de transmisión
for l in L, t in T
    @constraint(model, b_susceptance[l]*(theta[lines_df.BarIni[l], t] - theta[lines_df.BarFin[l], t]) <= line_max[l])
end

# Fijar el ángulo del nodo slack (nodo 1) a 0 para todos los periodos
for t in T
    @constraint(model, theta[1, t] == 0)
end

# Función objetivo: minimizar el costo total
@objective(model, Min, sum(gen_cost[g] * p[g, t] for g in G for t in T))

# Resolver el modelo
optimize!(model)

# Imprimir el valor de la función objetivo y la solución
valor_objetivo = objective_value(model)
println("Valor de la función objetivo: ", valor_objetivo*pot_base)

# Extraer e imprimir la solución
solucion_potencia = value.(p)*pot_base
solucion_angulos = value.(theta)
println("Potencia generada por generador y tiempo:")
println(solucion_potencia)
println("Ángulo del voltaje en nodo y tiempo:")
println(solucion_angulos)

# Imprimir los precios sombra de la restricción de demanda
println("Precios sombra de la demanda por nodo y tiempo:")
for n in N, t in T
    println("Precio sombra en nodo ", n, " en tiempo ", t, ": ", dual(demanda[n, t]))
end

# Crear matriz para almacenar los valores duales
precios_sombra = Array{Float64}(undef, length(N), length(T))

# Rellenar matriz con los precios sombra
for n in N
    for t in T
        precios_sombra[n, t] = dual(demanda[n, t])
    end
end

# Imprimir la matriz de precios sombra redondeada a dos decimales
println("Matriz de precios sombra (Nodos x Tiempos):")
for n in N
    for t in T
        print(round(precios_sombra[n, t], digits=2), " ")
    end
    println()  # Nueva línea para cada nodo
end

# Escribir los precios sombra en un archivo Excel
XLSX.openxlsx("precios_sombra_p1.xlsx", mode="w") do xf
    sheet = XLSX.addsheet!(xf, "Precios Sombra")
    for n in N
        for t in T
            XLSX.setdata!(sheet, XLSX.CellRef(n+1, t+1), round(precios_sombra[n, t], digits=2))
        end
    end
    # Agregar etiquetas de tiempo como cabecera
    for t in T
        XLSX.setdata!(sheet, XLSX.CellRef(1, t+1), "Tiempo $t")
    end
    # Agregar etiquetas de nodo como cabecera de fila
    for n in N
        XLSX.setdata!(sheet, XLSX.CellRef(n+1, 1), "Nodo $n")
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-20
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i9-10885H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 126 rows, 90 columns and 306 nonzeros
Model fingerprint: 0x66697547
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [3e+01, 4e+01]
  Bounds range     [2e-01, 6e+00]
  RHS range        [1e-01, 3e+00]
Presolve removed 125 rows and 87 columns
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0597500e+03   1.875000e-01   0.000000e+00      0s
       1    1.0635000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.063500000e+03

User-callback calls 

# Pregunta 1 (con rampa)

In [13]:
using JuMP, CSV, DataFrames, XLSX
using Gurobi  # Cambié GLPK por Gurobi, ya que lo mencionas en tu enunciado inicial

# Carga los datos desde los archivos CSV
demand_df = CSV.read("Demand.csv", DataFrame)
generators_df = CSV.read("Generators.csv", DataFrame)
lines_df = CSV.read("Lines.csv", DataFrame)

# Potencia base de 100MVA
pot_base = 100

# Conjunto de nodos, tiempos, generadores y líneas
N = 1:9 # Nodos de 1 a 9
T = 1:6 # Periodos de tiempo de t = 1 a t = 6
G = 1:size(generators_df, 1) # Generadores
L = 1:size(lines_df, 1) # Líneas de transmisión

# Inicializa el modelo de optimización
model = Model(Gurobi.Optimizer)

# Variables de decisión
@variable(model, p[g in G, t in T] >= 0) # Potencia generada por generador g en tiempo t
@variable(model, theta[n in N, t in T]) # Ángulo del voltaje en nodo n en tiempo t

# Parámetros del modelo
demand = Dict((n, t) => demand_df[n, t+1]/pot_base for n in N for t in T)
pot_min = Dict(g => generators_df.PotMin[g]/pot_base for g in G)
pot_max = Dict(g => generators_df.PotMax[g]/pot_base for g in G)
gen_cost = Dict(g => generators_df.GenCost[g] for g in G)
ramp = Dict(g => generators_df.Ramp[g]/pot_base for g in G)
line_max = Dict(l => lines_df.PotMax[l]/pot_base for l in L)
b_susceptance = Dict(l => 1/lines_df.Imp[l] for l in L)

# Restricciones del modelo
# Satisfacción de la demanda en cada nodo y tiempo con nombre para restricciones
@constraint(model, demanda[n in N, t in T],
    sum(p[g, t] for g in G if generators_df.BarConexion[g] == n) - 
    sum(b_susceptance[l]*(theta[lines_df.BarIni[l], t] - theta[lines_df.BarFin[l], t]) for l in L if lines_df.BarIni[l] == n) -
    sum(b_susceptance[l]*(theta[lines_df.BarFin[l], t] - theta[lines_df.BarIni[l], t]) for l in L if lines_df.BarFin[l] == n) == demand[n, t])

# Límites de generación de cada generador
for g in G, t in T
    @constraint(model, pot_min[g] <= p[g, t] <= pot_max[g])
end

# Capacidad de las líneas de transmisión
for l in L, t in T
    @constraint(model, b_susceptance[l]*(theta[lines_df.BarIni[l], t] - theta[lines_df.BarFin[l], t]) <= line_max[l])
end

# Restricciones de rampa
for g in G, t in 2:length(T)
    @constraint(model, -ramp[g] <= p[g, t] - p[g, t-1] <= ramp[g])
end

# Fijar el ángulo del nodo slack (nodo 1) a 0 para todos los periodos
for t in T
    @constraint(model, theta[1, t] == 0)
end

# Función objetivo: minimizar el costo total
@objective(model, Min, sum(gen_cost[g] * p[g, t] for g in G for t in T))

# Resolver el modelo
optimize!(model)

# Imprimir el valor de la función objetivo y la solución
valor_objetivo = objective_value(model)
println("Valor de la función objetivo: ", valor_objetivo*pot_base)

# Extraer e imprimir la solución
solucion_potencia = value.(p)*pot_base
solucion_angulos = value.(theta)
println("Potencia generada por generador y tiempo:")
println(solucion_potencia)
println("Ángulo del voltaje en nodo y tiempo:")
println(solucion_angulos)

# Imprimir los precios sombra de la restricción de demanda
println("Precios sombra de la demanda por nodo y tiempo:")
for n in N, t in T
    println("Precio sombra en nodo ", n, " en tiempo ", t, ": ", dual(demanda[n, t]))
end

# Crear matriz para almacenar los valores duales
precios_sombra = Array{Float64}(undef, length(N), length(T))

# Rellenar matriz con los precios sombra
for n in N
    for t in T
        precios_sombra[n, t] = dual(demanda[n, t])
    end
end

# Imprimir la matriz de precios sombra redondeada a dos decimales
println("Matriz de precios sombra (Nodos x Tiempos):")
for n in N
    for t in T
        print(round(precios_sombra[n, t], digits=2), " ")
    end
    println()  # Nueva línea para cada nodo
end

# Escribir los precios sombra en un archivo Excel
XLSX.openxlsx("precios_sombra_p1.xlsx", mode="w") do xf
    sheet = XLSX.addsheet!(xf, "Precios Sombra")
    for n in N
        for t in T
            XLSX.setdata!(sheet, XLSX.CellRef(n+1, t+1), round(precios_sombra[n, t], digits=2))
        end
    end
    # Agregar etiquetas de tiempo como cabecera
    for t in T
        XLSX.setdata!(sheet, XLSX.CellRef(1, t+1), "Tiempo $t")
    end
    # Agregar etiquetas de nodo como cabecera de fila
    for n in N
        XLSX.setdata!(sheet, XLSX.CellRef(n+1, 1), "Nodo $n")
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-20
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i9-10885H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 141 rows, 105 columns and 351 nonzeros
Model fingerprint: 0x54e25a89
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [3e+01, 4e+01]
  Bounds range     [2e-01, 6e+00]
  RHS range        [1e-01, 3e+00]
Presolve removed 120 rows and 74 columns
Presolve time: 0.00s
Presolved: 21 rows, 31 columns, 61 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0692020e+03   6.857350e+00   0.000000e+00      0s
      10    1.0872500e+03   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.087250000e+03

User-callback c

# Pregunta 2

In [14]:
using JuMP, CSV, DataFrames, XLSX
using Gurobi  # Utilizar Gurobi como solver

# Carga de datos
demand_df = CSV.read("Demand.csv", DataFrame)
generators_df = CSV.read("Generators.csv", DataFrame)
lines_df = CSV.read("Lines2.csv", DataFrame)

# Potencia base
pot_base = 100

# Conjuntos de nodos, periodos de tiempo, generadores y líneas
N = 1:9 # Nodos del 1 al 9
T = 1:6 # Periodos de tiempo de t = 1 a t = 6
G = 1:size(generators_df, 1) # Generadores
L = 1:size(lines_df, 1) # Líneas de transmisión

# Inicialización del modelo de optimización
model = Model(Gurobi.Optimizer)

# Variables de decisión
@variable(model, p[g in G, t in T] >= 0) # Potencia generada por cada generador g en el tiempo t
@variable(model, theta[n in N, t in T]) # Ángulo de voltaje en el nodo n en el tiempo t
@variable(model, p_insatisfecha[n in N, t in T] >= 0) # Potencia insatisfecha en el nodo n en el tiempo t

# Parámetros del modelo
demand = Dict((n, t) => demand_df[n, t+1]/pot_base for n in N for t in T)
pot_min = Dict(g => generators_df.PotMin[g]/pot_base for g in G)
pot_max = Dict(g => generators_df.PotMax[g]/pot_base for g in G)
gen_cost = Dict(g => generators_df.GenCost[g] for g in G)
ramp = Dict(g => generators_df.Ramp[g]/pot_base for g in G)
line_max = Dict(l => lines_df.PotMax[l]/pot_base for l in L)
b_susceptance = Dict(l => 1/lines_df.Imp[l] for l in L)

# Restricciones del modelo
@constraint(model, demanda[n in N, t in T], 
            sum(p[g, t] for g in G if generators_df.BarConexion[g] == n) - 
            sum(b_susceptance[l]*(theta[lines_df.BarIni[l], t] - theta[lines_df.BarFin[l], t]) for l in L if lines_df.BarIni[l] == n) -
            sum(b_susceptance[l]*(theta[lines_df.BarFin[l], t] - theta[lines_df.BarIni[l], t]) for l in L if lines_df.BarFin[l] == n) ==
            demand[n, t] - p_insatisfecha[n, t])

# Límites de generación
for g in G, t in T
    @constraint(model, pot_min[g] <= p[g, t] <= pot_max[g])
end

# Capacidad de las líneas de transmisión
for l in L, t in T
    @constraint(model, b_susceptance[l]*(theta[lines_df.BarIni[l], t] - theta[lines_df.BarFin[l], t]) <= line_max[l])
end

# Fijar el ángulo del nodo slack (nodo 1) a 0 para todos los periodos
for t in T
    @constraint(model, theta[1, t] == 0)
end

# Función objetivo: minimizar el costo total incluyendo multas por demanda insatisfecha
multa = 30  # $/MW
@objective(model, Min, sum(gen_cost[g] * p[g, t] for g in G for t in T) + multa * sum(p_insatisfecha[n, t] for n in N for t in T))

# Resolución del modelo
optimize!(model)

# Extracción e impresión de las soluciones prímales
solucion_potencia = value.(p)*pot_base
solucion_angulos = value.(theta)
solucion_insatisfecha = value.(p_insatisfecha)*pot_base
solucion_potencia_no_sat = value.(p_insatisfecha)*pot_base

# Imprimir el valor de la función objetivo
valor_objetivo = objective_value(model)
println("Valor de la función objetivo: ", valor_objetivo*pot_base)

# Impresión de las soluciones prímales
println("Potencia generada por generador y tiempo:")
println(solucion_potencia)
println("Potencia insatisfecha por nodo y tiempo:")
println(solucion_insatisfecha)

# Impresión de las soluciones duales (precios sombra)
println("Precios sombra de la demanda por nodo y tiempo:")
for n in N, t in T
    println("Precio sombra en nodo ", n, " en tiempo ", t, ": ", dual(demanda[n, t]))
end

# Crear matriz para almacenar los valores duales
precios_sombra = Array{Float64}(undef, length(N), length(T))

# Rellenar matriz con los precios sombra
for n in N
    for t in T
        precios_sombra[n, t] = dual(demanda[n, t])
    end
end

# Imprimir la matriz de precios sombra redondeada a dos decimales
println("Matriz de precios sombra (Nodos x Tiempos):")
for n in N
    for t in T
        print(round(precios_sombra[n, t], digits=2), " ")
    end
    println()  # Nueva línea para cada nodo
end

# Escribir los precios sombra en un archivo Excel
XLSX.openxlsx("precios_sombra_p2.xlsx", mode="w") do xf
    sheet = XLSX.addsheet!(xf, "Precios Sombra")
    for n in N
        for t in T
            XLSX.setdata!(sheet, XLSX.CellRef(n+1, t+1), round(precios_sombra[n, t], digits=2))
        end
    end
    # Agregar etiquetas de tiempo como cabecera
    for t in T
        XLSX.setdata!(sheet, XLSX.CellRef(1, t+1), "Tiempo $t")
    end
    # Agregar etiquetas de nodo como cabecera de fila
    for n in N
        XLSX.setdata!(sheet, XLSX.CellRef(n+1, 1), "Nodo $n")
    end
end

ArgumentError: ArgumentError: "Lines2.csv" is not a valid file or doesn't exist

# Pregunta 2 (con rampa)

In [15]:
using JuMP, CSV, DataFrames, XLSX
using Gurobi  # Utilizar Gurobi como solver

# Carga de datos
demand_df = CSV.read("Demand.csv", DataFrame)
generators_df = CSV.read("Generators.csv", DataFrame)
lines_df = CSV.read("Lines.csv", DataFrame)

# Potencia base
pot_base = 100

# Conjuntos de nodos, periodos de tiempo, generadores y líneas
N = 1:9 # Nodos del 1 al 9
T = 1:6 # Periodos de tiempo de t = 1 a t = 6
G = 1:size(generators_df, 1) # Generadores
L = 1:size(lines_df, 1) # Líneas de transmisión

# Inicialización del modelo de optimización
model = Model(Gurobi.Optimizer)

# Variables de decisión
@variable(model, p[g in G, t in T] >= 0) # Potencia generada por cada generador g en el tiempo t
@variable(model, theta[n in N, t in T]) # Ángulo de voltaje en el nodo n en el tiempo t
@variable(model, p_insatisfecha[n in N, t in T] >= 0) # Potencia insatisfecha en el nodo n en el tiempo t

# Parámetros del modelo
demand = Dict((n, t) => demand_df[n, t+1]/pot_base for n in N for t in T)
pot_min = Dict(g => generators_df.PotMin[g]/pot_base for g in G)
pot_max = Dict(g => generators_df.PotMax[g]/pot_base for g in G)
gen_cost = Dict(g => generators_df.GenCost[g] for g in G)
ramp = Dict(g => generators_df.Ramp[g]/pot_base for g in G)
line_max = Dict(l => lines_df.PotMax[l]/pot_base for l in L)
b_susceptance = Dict(l => 1/lines_df.Imp[l] for l in L)

# Restricciones del modelo
@constraint(model, demanda[n in N, t in T], 
            sum(p[g, t] for g in G if generators_df.BarConexion[g] == n) - 
            sum(b_susceptance[l]*(theta[lines_df.BarIni[l], t] - theta[lines_df.BarFin[l], t]) for l in L if lines_df.BarIni[l] == n) -
            sum(b_susceptance[l]*(theta[lines_df.BarFin[l], t] - theta[lines_df.BarIni[l], t]) for l in L if lines_df.BarFin[l] == n) ==
            demand[n, t] - p_insatisfecha[n, t])

# Límites de generación
for g in G, t in T
    @constraint(model, pot_min[g] <= p[g, t] <= pot_max[g])
end

# Capacidad de las líneas de transmisión
for l in L, t in T
    @constraint(model, b_susceptance[l]*(theta[lines_df.BarIni[l], t] - theta[lines_df.BarFin[l], t]) <= line_max[l])
end

# Restricciones de rampa
for g in G, t in 2:length(T)
    @constraint(model, -ramp[g] <= p[g, t] - p[g, t-1] <= ramp[g])
end

# Fijar el ángulo del nodo slack (nodo 1) a 0 para todos los periodos
for t in T
    @constraint(model, theta[1, t] == 0)
end

# Función objetivo: minimizar el costo total incluyendo multas por demanda insatisfecha
multa = 30  # $/MW
@objective(model, Min, sum(gen_cost[g] * p[g, t] for g in G for t in T) + multa * sum(p_insatisfecha[n, t] for n in N for t in T))

# Resolución del modelo
optimize!(model)

# Extracción e impresión de las soluciones prímales
solucion_potencia = value.(p)*pot_base
solucion_angulos = value.(theta)
solucion_insatisfecha = value.(p_insatisfecha)*pot_base
solucion_potencia_no_sat = value.(p_insatisfecha)*pot_base

# Imprimir el valor de la función objetivo
valor_objetivo = objective_value(model)
println("Valor de la función objetivo: ", valor_objetivo*pot_base)

# Impresión de las soluciones prímales
println("Potencia generada por generador y tiempo:")
println(solucion_potencia)
println("Potencia insatisfecha por nodo y tiempo:")
println(solucion_insatisfecha)

# Impresión de las soluciones duales (precios sombra)
println("Precios sombra de la demanda por nodo y tiempo:")
for n in N, t in T
    println("Precio sombra en nodo ", n, " en tiempo ", t, ": ", dual(demanda[n, t]))
end

# Crear matriz para almacenar los valores duales
precios_sombra = Array{Float64}(undef, length(N), length(T))

# Rellenar matriz con los precios sombra
for n in N
    for t in T
        precios_sombra[n, t] = dual(demanda[n, t])
    end
end

# Imprimir la matriz de precios sombra redondeada a dos decimales
println("Matriz de precios sombra (Nodos x Tiempos):")
for n in N
    for t in T
        print(round(precios_sombra[n, t], digits=2), " ")
    end
    println()  # Nueva línea para cada nodo
end

# Escribir los precios sombra en un archivo Excel
XLSX.openxlsx("precios_sombra_p2.xlsx", mode="w") do xf
    sheet = XLSX.addsheet!(xf, "Precios Sombra")
    for n in N
        for t in T
            XLSX.setdata!(sheet, XLSX.CellRef(n+1, t+1), round(precios_sombra[n, t], digits=2))
        end
    end
    # Agregar etiquetas de tiempo como cabecera
    for t in T
        XLSX.setdata!(sheet, XLSX.CellRef(1, t+1), "Tiempo $t")
    end
    # Agregar etiquetas de nodo como cabecera de fila
    for n in N
        XLSX.setdata!(sheet, XLSX.CellRef(n+1, 1), "Nodo $n")
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-20
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i9-10885H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 141 rows, 159 columns and 405 nonzeros
Model fingerprint: 0x243f66d9
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [3e+01, 4e+01]
  Bounds range     [2e-01, 6e+00]
  RHS range        [1e-01, 3e+00]
Presolve removed 111 rows and 107 columns
Presolve time: 0.00s
Presolved: 30 rows, 52 columns, 96 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.1975000e+02   8.909250e+00   0.000000e+00      0s
       8    1.0620000e+03   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.062000000e+03

User-callback c

# Pregunta 3

In [1]:
using JuMP, CSV, DataFrames, XLSX
using Gurobi # Utiliza el solver GLPK, puedes cambiarlo por otro si prefieres
using Plots

# Carga los datos desde los archivos CSV
demand_df = CSV.read("Demand.csv", DataFrame)
generators_df = CSV.read("Generators.csv", DataFrame)
lines_df = CSV.read("Lines.csv", DataFrame)
bess_df = CSV.read("Bess.csv", DataFrame)

# Potencia base de 100MVA
pot_base = 100

# Conjunto de nodos, tiempos, generadores y líneas
N = unique(demand_df[!, "IdBar"]) # Nodos de 1 a 9
T = 1:(size(demand_df, 2) - 1) # Periodos de tiempo de t = 1 a t = 6
G = 1:size(generators_df, 1) # Generadores
L = 1:size(lines_df, 1) # Líneas de transmisión

B = 1:size(bess_df, 1) # Sistemas de almacenamiento de baterías

# Inicializa el modelo de optimización
model = Model(Gurobi.Optimizer)

# Variables de decisión
@variable(model, p[g in G, t in T] >= 0) # Potencia generada por generador g en tiempo t
@variable(model, theta[n in N, t in T]) # Ángulo del voltaje en nodo n en tiempo t

# Variables de decisión para BESS
@variable(model, e[b in B, t in T] >= 0) # Energia guardada de la batería BESS b en tiempo t
@variable(model, dp[b in B, t in T] >= 0) # Potencia producida de la batería BESS b en tiempo t
@variable(model, ds[b in B, t in T] >= 0) # Potencia storage de la batería BESS b en tiempo t

# Parámetros del modelo
demand = Dict((n, t) => demand_df[n, t+1]/pot_base for n in N for t in T) # Demanda dividida por potencia base
pot_min = Dict(g => generators_df.PotMin[g]/pot_base for g in G) # Potencia mínima dividida por potencia base
pot_max = Dict(g => generators_df.PotMax[g]/pot_base for g in G) # Potencia máxima dividida por potencia base
gen_cost = Dict(g => generators_df.GenCost[g] for g in G) # Costo de generación
ramp = Dict(g => generators_df.Ramp[g]/pot_base for g in G) # Rampa dividida por potencia base
line_max = Dict(l => lines_df.PotMax[l]/pot_base for l in L) # Capacidad máxima de línea dividida por potencia base
reactance = Dict(l => lines_df.Imp[l] for l in L) # Reactancia
b_susceptance = Dict(l => 1/lines_df.Imp[l] for l in L) # Susceptancia de línea (recíproco de la reactancia)
Cap_bateria = Dict(b => bess_df.Cap[b]/pot_base for b in B) # Capacidad de baterias
Horas_bateria = Dict(b => bess_df.Horas[b] for b in B) # Horas de baterias
Rend_bateria = Dict(b => bess_df.Rend[b] for b in B) # Rendimiento de baterias
Einicial_bateria = Dict(b => bess_df.E_inicial[b] for b in B) # Energia inicial de baterias
Efinal_bateria = Dict(b => bess_df.E_final[b] for b in B) # Energia final de baterias
# Restricciones del modelo

# Satisfaccion de demanda
@constraint(model, demanda[n in N, t in T],
        sum(p[g, t] for g in G if generators_df.BarConexion[g] == n) +
        sum(dp[b, t] - ds[b, t] for b in B if bess_df.BarConexion[b] == n) -
        sum(b_susceptance[l] * (theta[lines_df.BarIni[l], t] - theta[lines_df.BarFin[l], t]) for l in L if lines_df.BarIni[l] == n) -
        sum(b_susceptance[l] * (theta[lines_df.BarFin[l], t] - theta[lines_df.BarIni[l], t]) for l in L if lines_df.BarFin[l] == n)
        == demand[n, t])

# Fijar el ángulo del nodo slack (nodo 1) a 0 para todos los periodos
for t in T
    @constraint(model, theta[1, t] == 0)
end

# Límites de generación de cada generador
for g in G, t in T
    @constraint(model, pot_min[g] <= p[g, t] <= pot_max[g])
end

# Restricciones de rampa
for g in G, t in 2:length(T)
   @constraint(model, -ramp[g] <= p[g, t] - p[g, t-1] <= ramp[g])
end

# Capacidad de las líneas de transmisión
for l in L, t in T
    @constraint(model, b_susceptance[l]*(theta[lines_df.BarIni[l], t] - theta[lines_df.BarFin[l], t]) <= line_max[l])
end

# Restricciones de las baterías BESS
for b in B, t in T
    # Restricciones para la capacidad de almacenamiento de la batería
    storage_capacity = Horas_bateria[b] * Cap_bateria[b]
    @constraint(model, e[b, t] <= storage_capacity)
    
    # Restricciones para la carga de la batería
    @constraint(model, ds[b, t] <= Cap_bateria[b])

    # Restricciones para la descarga de la batería
    @constraint(model, dp[b, t] <= Cap_bateria[b])

    # Dinámica de la batería
    if t>1
        @constraint(model, e[b, t] == e[b, t-1] + ds[b, t]*Rend_bateria[b] - dp[b, t]/Rend_bateria[b])
    elseif t==1
        @constraint(model, e[b, 1] == storage_capacity * Efinal_bateria[b] + ds[b, 1]*Rend_bateria[b] - dp[b, 1]/Rend_bateria[b])
    end
    if t == last(T)
        @constraint(model, e[b, t] == storage_capacity * Efinal_bateria[b])
    end
end

# Función objetivo: minimizar el costo total
@objective(model, Min, sum(gen_cost[g] * p[g, t] for g in G for t in T))

# Resolver el modelo
optimize!(model)

# Extraer la solución
solucion_potencia = value.(p)*pot_base
solucion_angulos = value.(theta)
solucion_potencia_producida_bateria = value.(dp)*pot_base
solucion_potencia_guardada_bateria = value.(ds)*pot_base
solucion_estadodecarga = value.(e)*pot_base
costo_total = objective_value(model)*pot_base


# Imprimir la solución
println("Potencia generada por generador y tiempo:")
println(solucion_potencia)

println("Ángulo del voltaje en nodo y tiempo:")
println(solucion_angulos)

println("Potencia guardada de la batería BESS:")
println(solucion_potencia_producida_bateria)

println("Potencia descargada de la batería BESS:")
println(solucion_potencia_guardada_bateria)

println("Energia de la batería BESS:")
println(solucion_estadodecarga)


println("Precios sombra de la demanda por nodo y tiempo:")
for n in N, t in T
    preciodual = dual(demanda[n, t])
    println("Precio sombra del nodo $n en el tiempo $t: $preciodual")
end
println("------------------------------------------------------------")
println("Costos totales de generación")
println(costo_total)


# Crear matriz para almacenar los valores duales
precios_sombra = Array{Float64}(undef, length(N), length(T))

# Rellenar matriz con los precios sombra
for n in N
    for t in T
        precios_sombra[n, t] = dual(demanda[n, t])
    end
end

# Imprimir la matriz de precios sombra redondeada a dos decimales
println("Matriz de precios sombra (Nodos x Tiempos):")
for n in N
    for t in T
        print(round(precios_sombra[n, t], digits=2), " ")
    end
    println()  # Nueva línea para cada nodo
end


Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-01
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.4.0 23E224)

CPU model: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 216 rows, 159 columns and 513 nonzeros
Model fingerprint: 0xc94c2d9f
Coefficient statistics:
  Matrix range     [8e-01, 4e+01]
  Objective range  [2e+01, 4e+01]
  Bounds range     [2e-01, 6e+00]
  RHS range        [1e-01, 3e+00]
Presolve removed 174 rows and 80 columns
Presolve time: 0.00s
Presolved: 42 rows, 79 columns, 177 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6729850e+02   1.094999e+01   0.000000e+00      0s
      43    1.0707776e+03   0.000000e+00   0.000000e+00      0s

Solved in 43 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.070777596e+03

User-callback calls 105, time in user-callbac